In [1]:
%%capture
!pip install pyspark
!pip install findspark
!pip install pyngrok

In [2]:
import findspark

findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession.builder \
        .appName('testColab') \
        .getOrCreate()

In [3]:
from pyngrok import ngrok, conf
import getpass

print("Enter your authtoken, which can be copied "
"from https://dashboard.ngrok.com/get-started/your-authtoken")
conf.get_default().auth_token = getpass.getpass()

ui_port = 4040
public_url = ngrok.connect(ui_port).public_url
print(f" * ngrok tunnel \"{public_url}\" -> \"http://127.0.0.1:{ui_port}\"")

Enter your authtoken, which can be copied from https://dashboard.ngrok.com/get-started/your-authtoken
··········


 * ngrok tunnel "https://ded3-35-196-89-184.ngrok-free.app" -> "http://127.0.0.1:4040"


In [12]:
from pyspark import SparkFiles

file_url = 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-10.parquet'
spark.sparkContext.addFile(file_url)

df = spark.read.parquet(SparkFiles.get('yellow_tripdata_2024-10.parquet'),
      header=True)

df.count()

3833771

In [15]:
df.repartition(4).write.mode('overwrite').save("dataframe.parquet",format='parquet')

In [50]:
from pathlib import Path

folder = Path('dataframe')  # replace with your folder path

# This will list only files ending with .parquet in the specified folder
for file in folder.glob('*.parquet'):
    size_bytes = file.stat().st_size/1024/1024
    print(f"{file.name}: {int(size_bytes)} MB")

part-00000-09339b86-8da9-4a5b-802d-3fbe13fd43f5-c000.snappy.parquet: 15 MB
part-00003-09339b86-8da9-4a5b-802d-3fbe13fd43f5-c000.snappy.parquet: 15 MB
part-00001-09339b86-8da9-4a5b-802d-3fbe13fd43f5-c000.snappy.parquet: 15 MB
part-00002-09339b86-8da9-4a5b-802d-3fbe13fd43f5-c000.snappy.parquet: 15 MB


In [18]:
df = df.withColumn('date_pickup', df['tpep_pickup_datetime'].cast('date'))

In [22]:
df_filtered = df.filter("date_pickup = '2024-10-15'")

In [24]:
#Number of trips in October 15th
df_filtered.count()

128893

In [25]:
df.show(5)

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|date_pickup|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+-----------+
|       2| 2024-10-01 00:30:44|  2024-10-01 00:48:26|              1|          3.0|         1|                 N|         162|         246|           1

In [29]:
from pyspark.sql.functions import *
df_longest_trip = df.withColumn('hours_diff',(unix_timestamp("tpep_dropoff_datetime")-unix_timestamp("tpep_pickup_datetime"))/3600)

In [33]:
#Longest trip in hours
df_longest_trip.groupby().max('hours_diff').first()

Row(max(hours_diff)=162.61777777777777)

In [34]:
file_url = 'https://d37ci6vzurychx.cloudfront.net/misc/taxi_zone_lookup.csv'
spark.sparkContext.addFile(file_url)

df_zones = spark.read.csv(SparkFiles.get('taxi_zone_lookup.csv'),
      header=True)

In [41]:
df_zones_count = df.join(df_zones, df.PULocationID == df_zones.LocationID).select(df_zones.Zone).groupby('Zone').count().sort(asc('count'))
df_zones_count.first().asDict()

{'Zone': "Governor's Island/Ellis Island/Liberty Island", 'count': 1}